# Multiple Inheritance in Python
## Stuff we need to know:
### What does super() do?
### What is MRO?

In [1]:
# This is our base class its already inheriting from object but we can omit that if we want

class Base(object):
    def __init__(self):
        print('Enter Base.__init__')
        self.base_message = 'Hello from base'
        print('Exit Base.__init__')

In [2]:
# This is a child class it inherits from the base class and we get a first look at the super() function

class ChildA(Base):
    def __init__(self):
        print('Enter ChildA.__init__')
        super().__init__()
        self.child_message = 'Hello from child A'
        print('Exit ChildA.__init__')

In [3]:
my_class = ChildA()
print(my_class.base_message)
print(my_class.child_message)

Enter ChildA.__init__
Enter Base.__init__
Exit Base.__init__
Exit ChildA.__init__
Hello from base
Hello from child A


Make note of the order here. This is important.

### Hey, you said we were doing multiple inheritance!

Okay, time to define another class that will add to the child.

In [12]:
class Other(object):
    def __init__(self):
        print('Enter Other.__init__')
        self.other_message = "Hello from other"
        print('Exit Other.__init__')

In [13]:
class ChildB(Base, Other):
    def __init__(self):
        print('Enter ChildB.__init__')
        super().__init__()
        self.child_message = 'Hello from child B'
        print('Exit ChildB.__init__')

In [14]:
my_class = ChildB()
print(my_class.base_message)
print(my_class.child_message)
print(my_class.other_message)

Enter ChildB.__init__
Enter Base.__init__
Exit Base.__init__
Exit ChildB.__init__
Hello from base
Hello from child B


AttributeError: 'ChildB' object has no attribute 'other_message'

Okay so that did not work. Lets try again.

In [15]:
class ChildC(Other, Base):
    def __init__(self):
        print('Enter ChildC.__init__')
        super().__init__()
        self.child_message = 'Hello from child C'
        print('Exit ChildC.__init__')

my_class = ChildC()
print(my_class.base_message)
print(my_class.child_message)
print(my_class.other_message)

Enter ChildC.__init__
Enter Other.__init__
Exit Other.__init__
Exit ChildC.__init__


AttributeError: 'ChildC' object has no attribute 'base_message'

So whats going on?

### MRO

We need to look at method resolution order:


In [16]:
print(f"MRO of B: {ChildB.__mro__}")
print(f"MRO of C: {ChildC.__mro__}")

MRO of B: (<class '__main__.ChildB'>, <class '__main__.Base'>, <class '__main__.Other'>, <class 'object'>)
MRO of C: (<class '__main__.ChildC'>, <class '__main__.Other'>, <class '__main__.Base'>, <class 'object'>)


So the `super()` is calling the MRO tree in order.
Great lets add another!

In [17]:
class ChildD(Other, Base):
    def __init__(self):
        print('Enter ChildD.__init__')
        super().__init__()
        super().__init__()
        self.child_message = 'Hello from child D'
        print('Exit ChildD.__init__')

my_class = ChildD()
print(my_class.base_message)
print(my_class.child_message)
print(my_class.other_message)

Enter ChildD.__init__
Enter Other.__init__
Exit Other.__init__
Enter Other.__init__
Exit Other.__init__
Exit ChildD.__init__


AttributeError: 'ChildD' object has no attribute 'base_message'

But it again does not work. 

The when in the `__init__` of D the next item in the MRO is other so super calls `Other.__init__` both times

#### Making it work

In [20]:
class SuperOther(object):
    def __init__(self):
        print('Enter SuperOther.__init__')
        self.other_message = "Hello from super-other"
        super().__init__()
        print('Exit SuperOther.__init__')

class ChildE(SuperOther, Base):
    def __init__(self):
        print('Enter ChildE.__init__')
        super().__init__()
        self.child_message = 'Hello from child D'
        print('Exit ChildE.__init__')

my_class = ChildE()
print(my_class.base_message)
print(my_class.child_message)
print(my_class.other_message)

Enter ChildE.__init__
Enter SuperOther.__init__
Enter Base.__init__
Exit Base.__init__
Exit SuperOther.__init__
Exit ChildE.__init__
Hello from base
Hello from child D
Hello from super-other


Working but not ideal we would rather have base as the fist argument to establish that other is adding some stuff

In [21]:
class ChildF(Base, SuperOther):
    def __init__(self):
        print('Enter ChildF.__init__')
        super().__init__()
        self.child_message = 'Hello from child F'
        print('Exit ChildF.__init__')

my_class = ChildF()
print(my_class.base_message)
print(my_class.child_message)
print(my_class.other_message)

Enter ChildF.__init__
Enter Base.__init__
Exit Base.__init__
Exit ChildF.__init__
Hello from base
Hello from child F


AttributeError: 'ChildF' object has no attribute 'other_message'

But now it stops working again. 

You can add a debug point to the above and step though but from the above we can see that we enter and exit Base.

Base has no super, it's a base. 

In this case, we could:

In [26]:
class ChildG(Base, Other):
    def __init__(self):
        print('Enter ChildG.__init__')
        super().__init__()
        Other.__init__(self)
        self.child_message = 'Hello from child G'
        print('Exit ChildG.__init__')


my_class = ChildG()
print(my_class.base_message)
print(my_class.child_message)
print(my_class.other_message)

Enter ChildG.__init__
Enter Base.__init__
Exit Base.__init__
Enter Other.__init__
Exit Other.__init__
Exit ChildG.__init__
Hello from base
Hello from child G
Hello from other


Thats not all.

The MRO and the position of supers determines what does or does not get initialized in `__init__`. We can be picky using named parent classes. However then things inheriting our class will break if they use a super. 

For completeness lets check what happens if we use SuperOther after Base.

In [27]:
class ChildG(Base, SuperOther):
    def __init__(self):
        print('Enter ChildG.__init__')
        super().__init__()
        SuperOther.__init__(self)
        self.child_message = 'Hello from child G'
        print('Exit ChildG.__init__')

my_class = ChildG()
print(my_class.base_message)
print(my_class.child_message)
print(my_class.other_message)

Enter ChildG.__init__
Enter Base.__init__
Exit Base.__init__
Enter SuperOther.__init__
Exit SuperOther.__init__
Exit ChildG.__init__
Hello from base
Hello from child G
Hello from super-other


Same behavior, unless `SuperOther` has it's own parents then it would follow that path for a bit.

Lets do a final check on one thing:

In [29]:
class SuperBase(object):
    def __init__(self):
        print('Enter SuperBase.__init__')
        self.base_message = 'Hello from super base'
        super().__init__()
        print('Exit SuperBase.__init__')

class ChildH(SuperBase, Other):
    def __init__(self):
        print('Enter ChildH.__init__')
        super().__init__()
        self.child_message = 'Hello from child H'
        print('Exit ChildH.__init__')

my_class = ChildH()
print(my_class.base_message)
print(my_class.child_message)
print(my_class.other_message)

Enter ChildH.__init__
Enter SuperBase.__init__
Enter Other.__init__
Exit Other.__init__
Exit SuperBase.__init__
Exit ChildH.__init__
Hello from super base
Hello from child H
Hello from other


By putting a `super()` call in base it manages to reach `Other`

Check out the MRO.

In [30]:
print(f"MRO of H: {ChildH.__mro__}")

MRO of H: (<class '__main__.ChildH'>, <class '__main__.SuperBase'>, <class '__main__.Other'>, <class 'object'>)


Fantastic but why did it not call object first?

MRO puts the 'lowest common denominator' at the end of the resolution. All our classes relied on object so it goes last!

In [33]:
class ObjectlessOther():
    def __init__(self):
        print('Enter ObjectlessOther.__init__')
        self.other_message = "Hello from objectless other"
        print('Exit ObjectlessOther.__init__')

class ChildI(SuperBase, ObjectlessOther):
    def __init__(self):
        print('Enter ChildI.__init__')
        super().__init__()
        self.child_message = 'Hello from child I'
        print('Exit ChildI.__init__')

my_class = ChildI()
print(my_class.base_message)
print(my_class.child_message)
print(my_class.other_message)
print(f"MRO of I: {ChildI.__mro__}")
print(f"MRO of ObjectlessOther: {ObjectlessOther.__mro__}")

Enter ChildI.__init__
Enter SuperBase.__init__
Enter ObjectlessOther.__init__
Exit ObjectlessOther.__init__
Exit SuperBase.__init__
Exit ChildI.__init__
Hello from super base
Hello from child I
Hello from objectless other
MRO of I: (<class '__main__.ChildI'>, <class '__main__.SuperBase'>, <class '__main__.ObjectlessOther'>, <class 'object'>)
MRO of ObjectlessOther: (<class '__main__.ObjectlessOther'>, <class 'object'>)


Even a Class without object has object!


## Reviewing our learning objectives
### What does super() do?

`super` is a super way to call the next item in the Method Resolution Order (MRO)

### What is MRO?

MRO is method resolution order, or the order in which Python will look for items. 

## All together

As a combo super and MRO are commonly utilized in Classes in `__init__` (or 'dunder') methods when one class wants to access the `__init__` of it's parent, or as we have shown parents.

The MRO is determined by the 'C3 Linearization algorithm'.

"Basically, the idea behind C3 is that if you write down all of the ordering rules imposed by inheritance relationships in a complex class hierarchy, the algorithm will determine a monotonic ordering of the classes that satisfies all of them. If such an ordering can not be determined, the algorithm will fail." - [GVR](http://python-history.blogspot.com/2010/06/method-resolution-order.html)

# Wait you said commonly utilized and dunder

Yes.

Super is far more powerful and if we are writing a package of significant complexity then we should know more.

Next notebook please.